In [45]:
import os
import time
import math

import pandas as pd
import tenseal as ts

In [59]:
CKKS_POLY_DEGREE = 16384
BFV_POLY_DEGREE  = 16384

CKKS_COEFF_BITS = (60, 40, 40, 40, 60)
CKKS_SCALE = 2**40

BFV_PLAIN_MODULUS = 1099511922689
BFV_MAX_SAFE_SLOTS = 4096

In [60]:
def setup_ckks(poly_modulus_degree=CKKS_POLY_DEGREE, coeff_mod_bit_sizes=CKKS_COEFF_BITS, scale=CKKS_SCALE):
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=poly_modulus_degree,
        coeff_mod_bit_sizes=list(coeff_mod_bit_sizes),
    )
    context.global_scale = scale
    context.generate_galois_keys()
    return context


def setup_bfv(poly_modulus_degree=BFV_POLY_DEGREE, plain_modulus=BFV_PLAIN_MODULUS):
    context = ts.context(
        ts.SCHEME_TYPE.BFV,
        poly_modulus_degree=poly_modulus_degree,
        plain_modulus=plain_modulus,
    )
    context.generate_galois_keys()
    context.generate_relin_keys()
    return context

def ckks_max_slots():
    return CKKS_POLY_DEGREE // 2

def bfv_max_slots():
    return BFV_MAX_SAFE_SLOTS

def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

def holder_encrypt_ckks(context, data):
    chunks = list(chunk_list(data, ckks_max_slots()))
    return [ts.ckks_vector(context, c) for c in chunks]


def holder_encrypt_bfv(context, data):
    chunks = list(chunk_list(data, bfv_max_slots()))
    return [ts.bfv_vector(context, c) for c in chunks]

def analyzer_process_ckks_addition(enc_chunks):
    partial = [c.sum() for c in enc_chunks]
    acc = partial[0]
    for p in partial[1:]:
        acc += p
    return acc


def analyzer_process_bfv_addition(enc_chunks):
    partial = [c.sum() for c in enc_chunks]
    acc = partial[0]
    for p in partial[1:]:
        acc += p
    return acc

def analyzer_process_ckks_multiplication(enc_chunks):
    partial = [(c * 2).sum() for c in enc_chunks]
    acc = partial[0]
    for p in partial[1:]:
        acc += p
    return acc


def analyzer_process_bfv_multiplication(enc_chunks):
    partial = [(c * 2).sum() for c in enc_chunks]
    acc = partial[0]
    for p in partial[1:]:
        acc += p
    return acc

def holder_decrypt_scalar(enc_result):
    return enc_result.decrypt()[0]


In [61]:
def benchmark(dataset, ctx_ckks, ctx_bfv):
    gt_sum = sum(dataset)
    gt_mul = sum(x * 2 for x in dataset)

    print("\n" + "=" * 80)
    print(f"DATASET SIZE: {len(dataset)}")
    print("=" * 80)

    # ======================================================
    # CKKS
    # ======================================================
    ckks_slots = ckks_max_slots()
    ckks_chunks = math.ceil(len(dataset) / ckks_slots)

    print("\nCKKS PARAMETERS")
    print(f"  poly_modulus_degree: {CKKS_POLY_DEGREE}")
    print(f"  max slots:           {ckks_slots}")
    print(f"  chunks:              {ckks_chunks}")

    # ---------------- CKKS ADDITION ----------------
    print("\n--- CKKS ADDITION ---")

    t0_total = time.time()

    t0 = time.time()
    enc_ckks = holder_encrypt_ckks(ctx_ckks, dataset)
    t_enc = time.time() - t0

    t0 = time.time()
    res_ckks = analyzer_process_ckks_addition(enc_ckks)
    t_proc = time.time() - t0

    t0 = time.time()
    dec_ckks = holder_decrypt_scalar(res_ckks)
    t_dec = time.time() - t0

    t_total = time.time() - t0_total

    print(f"Result:      {dec_ckks:.2f}")
    print(f"GroundTruth: {gt_sum}")
    print(f"Error:       {abs(dec_ckks - gt_sum):.4f}")
    print(f"Encryption:  {t_enc:.4f}s")
    print(f"Processing:  {t_proc:.4f}s")
    print(f"Decryption:  {t_dec:.4f}s")
    print(f"Total Time:  {t_total:.4f}s")

    # ---------------- CKKS MULTIPLICATION ----------------
    print("\n--- CKKS MULTIPLICATION ---")

    t0_total = time.time()

    t0 = time.time()
    enc_ckks = holder_encrypt_ckks(ctx_ckks, dataset)
    t_enc = time.time() - t0

    t0 = time.time()
    res_ckks = analyzer_process_ckks_multiplication(enc_ckks)
    t_proc = time.time() - t0

    t0 = time.time()
    dec_ckks = holder_decrypt_scalar(res_ckks)
    t_dec = time.time() - t0

    t_total = time.time() - t0_total

    print(f"Result:      {dec_ckks:.2f}")
    print(f"GroundTruth: {gt_mul}")
    print(f"Error:       {abs(dec_ckks - gt_mul):.4f}")
    print(f"Encryption:  {t_enc:.4f}s")
    print(f"Processing:  {t_proc:.4f}s")
    print(f"Decryption:  {t_dec:.4f}s")
    print(f"Total Time:  {t_total:.4f}s")

    # ======================================================
    # BFV
    # ======================================================
    bfv_slots = bfv_max_slots()
    bfv_chunks = math.ceil(len(dataset) / bfv_slots)

    print("\nBFV PARAMETERS")
    print(f"  poly_modulus_degree: {BFV_POLY_DEGREE}")
    print(f"  max slots:           {bfv_slots}")
    print(f"  chunks:              {bfv_chunks}")

    # ---------------- BFV ADDITION ----------------
    print("\n--- BFV ADDITION ---")

    t0_total = time.time()

    t0 = time.time()
    enc_bfv = holder_encrypt_bfv(ctx_bfv, dataset)
    t_enc = time.time() - t0

    t0 = time.time()
    res_bfv = analyzer_process_bfv_addition(enc_bfv)
    t_proc = time.time() - t0

    t0 = time.time()
    dec_bfv = holder_decrypt_scalar(res_bfv)
    t_dec = time.time() - t0

    t_total = time.time() - t0_total

    print(f"Result:      {dec_bfv}")
    print(f"GroundTruth: {gt_sum}")
    print(f"Error:       {abs(dec_bfv - gt_sum)}")
    print(f"Encryption:  {t_enc:.4f}s")
    print(f"Processing:  {t_proc:.4f}s")
    print(f"Decryption:  {t_dec:.4f}s")
    print(f"Total Time:  {t_total:.4f}s")

    # ---------------- BFV MULTIPLICATION ----------------
    print("\n--- BFV MULTIPLICATION ---")

    t0_total = time.time()

    t0 = time.time()
    enc_bfv = holder_encrypt_bfv(ctx_bfv, dataset)
    t_enc = time.time() - t0

    t0 = time.time()
    res_bfv = analyzer_process_bfv_multiplication(enc_bfv)
    t_proc = time.time() - t0

    t0 = time.time()
    dec_bfv = holder_decrypt_scalar(res_bfv)
    t_dec = time.time() - t0

    t_total = time.time() - t0_total

    print(f"Result:      {dec_bfv}")
    print(f"GroundTruth: {gt_mul}")
    print(f"Error:       {abs(dec_bfv - gt_mul)}")
    print(f"Encryption:  {t_enc:.4f}s")
    print(f"Processing:  {t_proc:.4f}s")
    print(f"Decryption:  {t_dec:.4f}s")
    print(f"Total Time:  {t_total:.4f}s")


In [62]:
df = pd.read_csv("datasets/dataset_131072.csv")
salaries = df["salary_cents"].tolist()

ctx_ckks = setup_ckks()
ctx_bfv = setup_bfv()

for size in [4096, 8192, 16384, 32768, 65536, 131072]:
    benchmark(salaries[:size], ctx_ckks, ctx_bfv)


DATASET SIZE: 4096

CKKS PARAMETERS
  poly_modulus_degree: 16384
  max slots:           8192
  chunks:              1

--- CKKS ADDITION ---
Result:      1126380516.00
GroundTruth: 1126380516
Error:       0.0000
Encryption:  0.0216s
Processing:  0.1003s
Decryption:  0.0035s
Total Time:  0.1254s

--- CKKS MULTIPLICATION ---
Result:      2252764254.60
GroundTruth: 2252761032
Error:       3222.6031
Encryption:  0.0107s
Processing:  0.0729s
Decryption:  0.0038s
Total Time:  0.0875s

BFV PARAMETERS
  poly_modulus_degree: 16384
  max slots:           4096
  chunks:              1

--- BFV ADDITION ---
Result:      1126380516
GroundTruth: 1126380516
Error:       0
Encryption:  0.0321s
Processing:  0.2701s
Decryption:  0.0053s
Total Time:  0.3074s

--- BFV MULTIPLICATION ---
Result:      2252761032
GroundTruth: 2252761032
Error:       0
Encryption:  0.0133s
Processing:  0.2519s
Decryption:  0.0059s
Total Time:  0.2711s

DATASET SIZE: 8192

CKKS PARAMETERS
  poly_modulus_degree: 16384
  max sl